# TensorTrade - Renderers and Plotly Visualization Chart
## Data Loading Function

In [ ]:
# ipywidgets is required to run Plotly in Jupyter Notebook.
# Uncomment and run the following line to install it if required.

#!pip install ipywidgets

In [1]:
import ta

import pandas as pd

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC
from tensortrade.oms.wallets import Wallet, Portfolio

#%matplotlib inline

In [ ]:
def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=1)
    df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp form "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
    df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    # Convert the date column type from string to datetime for proper sorting.
    df['date'] = pd.to_datetime(df['date'])

    # Make sure historical prices are sorted chronologically, oldest first.
    df.sort_values(by='date', ascending=True, inplace=True)

    df.reset_index(drop=True, inplace=True)

    # Format timestamps as you want them to appear on the chart buy/sell marks.
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')

    return df

In [ ]:
df = load_csv('Coinbase_BTCUSD_1h.csv')
df.head()

In [2]:
#df2 = load_csv('Bitstamp_BTCUSD_1h.csv')
#df2.head()
cdd = CryptoDataDownload()

df = cdd.fetch("Bitstamp", "USD", "BTC", "1h")
df.head()

,date,unix,open,high,low,close,volume
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,559.93
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,273.58
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,917.79
3,2018-05-15 09:00:00,1526374800,8728.49,8754.40,8701.35,8708.32,182.62
4,2018-05-15 10:00:00,1526378400,8708.32,8865.00,8695.11,8795.90,1260.69


## Data Preparation
### Create the dataset features

In [3]:
dataset = ta.add_all_ta_features(df, 'open', 'high', 'low', 'close', 'volume', fillna=True)
dataset.head(3)

#recent Bitstamp data
#data_set = ta.add_all_ta_features(data, 'open', 'high', 'low', 'close', 'volume', fillna=True)
#data_set.head()

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



,date,unix,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,559.93,-137.665116,559.93,-0.245861,...,-62.293065,0.0,8740.990000,0.0,0.000000,0.000000,0.000000,3.600372,0.000000,0.000000
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,273.58,-193.185584,286.35,-0.231774,...,-64.519016,0.0,8740.103806,0.0,-4.240196,-0.848039,-3.392157,-0.022766,-0.022769,-0.022766
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,917.79,279.107064,-631.44,0.159371,...,-50.084466,0.0,8734.893333,0.0,1.924992,-0.293433,2.218425,-0.120265,-0.120338,-0.143004


### Create Chart Price History Data
Note: It is recommended to create the chart data *after* creating and cleaning the dataset to ensure one-to-one mapping between the historical prices data and the dataset.

In [4]:
price_history = dataset[['date', 'open', 'high', 'low', 'close', 'volume']]  # chart data
display(price_history.head(3))

dataset.drop(columns=['date', 'open', 'high', 'low', 'close', 'volume'], inplace=True)

,date,open,high,low,close,volume
0,2018-05-15 06:00:00,8733.86,8796.68,8707.28,8740.99,559.93
1,2018-05-15 07:00:00,8740.99,8766.00,8721.11,8739.00,273.58
2,2018-05-15 08:00:00,8739.00,8750.27,8660.53,8728.49,917.79


In [ ]:
price_history2 = data_set[['date', 'open', 'high', 'low', 'close', 'volume']]  # chart data
display(price_history2.head(3))

data_set.drop(columns=['date', 'open', 'high', 'low', 'close', 'volume'], inplace=True)

In [ ]:
price_history.tail()

## Setup Trading Environment
### Create Data Feeds

In [10]:
bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(price_history['close'].tolist(), dtype="float").rename("USD-BTC")
)

#bitstamp = Exchange("bitstamp", service=execute_order)(
#    Stream.source(price_history2['close'].tolist(), dtype="float").rename("USD-BTC")
#)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC),
])

# portfolio2 = Portfolio(USD, [
#     Wallet(coinbase, 10000 * USD),
#     Wallet(coinbase, 10 * BTC),
# ])

with NameSpace("bitstamp"):
    streams = [Stream.source(dataset[c].tolist(), dtype="float").rename(c) for c in dataset.columns]

# with NameSpace("bitstamp"):
#     streams2 = [Stream.source(data_set[c].tolist(), dtype="float").rename(c) for c in data_set.columns]

feed = DataFeed(streams)
#feed2 = DataFeed(streams2)

#feed2.next()

In [ ]:
feed.next()

### Trading Environment Renderers
A renderer is a channel for the trading environment to output its current state. One or more renderers can be attached to the environment at the same time. For example, you can let the environment draw a chart and log to a file at the same time.

Notice that while all renderers can technically be used together, you need to select the best combination to avoid undesired results. For example, PlotlyTradingChart can work well with FileLogger but may not display well with ScreenLogger.

Renderer can be set by name (string) or class, single or list. Available renderers are:
* `'screenlog'` or `ScreenLogger`: Shows results on the screen.
* `'filelog'` or `FileLogger`: Logs results to a file.
* `'plotly'` or `PlotlyTradingChart`: A trading chart based on Plotly.

#### Examples:

* renderers = 'screenlog' (default)
* renderers = ['screenlog', 'filelog']
* renderers = ScreenLogger()
* renderers = ['screenlog', `FileLogger()`]
* renderers = [`FileLogger(filename='example.log')`]

Renderers can also be created and configured first then attached to the environment as seen in a following example.

### Trading Environment with a Single Renderer

In [6]:
import tensortrade.env.default as default

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer="screen-log",  # ScreenLogger used with default settings
    window_size=40
)

In [ ]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)
agent.train(n_episodes=4, n_steps=1000, render_interval=500)

### Environment with Multiple Renderers
Create PlotlyTradingChart and FileLogger renderers. Configuring renderers is optional as they can be used with their default settings.

In [11]:
from tensortrade.env.default.renderers import PlotlyTradingChart, FileLogger

chart_renderer = PlotlyTradingChart(
    display=True,  # show the chart on screen (default)
    height=800,  # affects both displayed and saved file height. None for 100% height.
    save_format="html",  # save the chart to an HTML file
    auto_open_html=True,  # open the saved HTML chart in a new browser tab
)

file_logger = FileLogger(
    filename="example.log",  # omit or None for automatic file name
    path="training_logs"  # create a new directory if doesn't exist, None for no directory
)

### Environement with Multiple Renderers

With the plotly renderer you must provide an parameter called `renderer_feed`. This is a `DataFeed` instance that provides all the information that is required by a renderer to function.

In [12]:
renderer_feed = DataFeed([
    Stream.source(price_history[c].tolist(), dtype="float").rename(c) for c in price_history]
)

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    window_size=40,
    renderer_feed=renderer_feed,
    renderer=chart_renderer
)

In [ ]:
renderer_feed2 = DataFeed([
    Stream.source(price_history2[c].tolist(), dtype="float").rename(c) for c in price_history2]
)

env2 = default.create(
    portfolio=portfolio2,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed2,
    window_size=40,
    renderer_feed=renderer_feed2,
    renderer=chart_renderer
)

In [ ]:
price_history2.head()

In [ ]:
print(portfolio)

In [ ]:
pip install ipywidgets

## Setup and Train DQN Agent
The green and red arrows shown on the chart represent buy and sell trades respectively. The head of each arrow falls at the trade execution price.

In [ ]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)
agent.train(n_episodes=4, n_steps=5000, render_interval=2500)

====      AGENT ID: 2a182378-ac84-444e-a864-c0eec9e873d5      ====


In [ ]:
from tensortrade.agents import DQNAgent

agent2 = DQNAgent(env2)
agent2.train(n_episodes=4, n_steps=1000, render_interval=500)

In [ ]:
from tensortrade.agents import A2CAgent

agentA2C = A2CAgent(env)
agentA2C.train(n_episodes=4, n_steps=1000, render_interval=500)

## Direct Performance and Net Worth Plotting
Alternatively, the final performance and net worth can be displayed using pandas via Matplotlib.

In [ ]:
#%matplotlib inline

portfolio.performance.plot()

In [ ]:
portfolio.performance.net_worth.plot()